# **Thompson sampling**(TS)によるバンディットのアルゴリズムを考えていきます．

今回の実験は報酬は確率的に作られていく**確率バンディット**を考えていきます.

**確率バンディット**とはアームiを引いた時に発生される報酬 $X_{i}$ は $P_{i}$ の分布に従って生成されるというものです．

TSでは報酬の期待値 $\mu_{i}$ は何らかのモデルの事前分布 $\pi_{i}(\mu_{i})$ から生成されていると仮定します．

つまり, **報酬の期待値の情報が含まれている事前分布がうまく表現できていれば, 報酬の分布との積分をとり事後分布でそのアームが良いものなのかを確認できるということです．**

----

### 実験設定を説明していきます．
$\pi_{i}(\mu_{i})$ は**ベータ分布**に従うと仮定します.

ベータ分布は,
$$f(X; \alpha, \beta) = \frac{X^{\alpha-1}(1-X)^{\beta-1}}{B(\alpha, \beta)}$$

$X$: 確率変数, 
$\alpha$: パラメータ, 
$\beta$: パラメータ, 

$B(\alpha, \beta)$ ベータ関数：
$$B(\alpha, \beta) = \int_0^1 X^{\alpha-1}(1-X)^{\beta-1} dx$$


報酬はベータ分布の共役分布である**ベルヌーイ分布**を想定します．
$$P(X=k) = \binom{n}{k}p^k(1-p)^{n-k}$$

$X$: 確率変数, 
$k$: 結果が1になる回数, 
$n$: 試行回数, 
$p$: 成功確率


---

回数tまでの観測された報酬の情報を$H_{t} = {X_{i(s)}}_{s=1}^{t-1}$ とします．

これらの表記を使い, **期待値のモデルの事後分布**を考えます．

t回のうちアームiを$n_{i} = N_{i}(t)$回引いたとし, 報酬1が$m$回, 報酬0が$n_{i}-m$が発生したとすると, **モデル$\pi_{i}(\mu_{i})$の事後分布**は次のようになります.


$$\pi_{i}(\mu_{i}|H_{t}) = Beta(\alpha+m, \beta+n_{i}-m)$$

となります．


そして, バンディットのアルゴリズムを思い出すと, 1番報酬が高そうなアームを引くことがゴールでした．
ここでアーム$i$が最善であることを仮定すると上の式はこのように書けます．
$$
\pi(\mu_{i}|H(t)) = \int_{0}^{1}\pi_{i}(x_{i}|H(t))(\prod_{j\ne i}\int_{0}^{x_{i}}\pi_{j}(x_{j}|H(t))dx_{j})dx_{i}
$$


上の式の直感的な意味としては, t回が終わった時に**アーム$i$の期待値はそれ以外のアームよりも高くなっていて欲しい**という意味です

ここでなぜ積分範囲が[0,1]かというと, 今回の報酬には**ベルヌーイ分布を想定しており, ベルヌーイ分布の期待値は[0,1]に収まる**ためです.

しかしながら, 上の計算は**一般的に困難**とされています.
よって次の代替案を適用することにします.

1) $\hat\mu_{i}$ を $\pi(\mu_{i}|H(t))$ に従って生成します．
2) $\hat\mu_{i}$ を最大にするアーム$i$を引きます．

----
### ここからアルゴリズムの説明にはいっていきます．


In [1]:
import random
import numpy as np

class Bandit:
    def __init__(self, n_bandits, epsilon):
        self.n_bandits = n_bandits
        self.epsilon = epsilon
        self.Q = np.zeros(n_bandits)
        self.N = np.zeros(n_bandits)
        self.bandits = np.random.normal(0, 1, (n_bandits,))
        
    def choose_action(self):
        if random.random() < self.epsilon:
            action = random.randint(0, self.n_bandits - 1)
        else:
            action = np.argmax(self.Q)
        return action
    
    def update(self, action, reward):
        self.N[action] += 1
        self.Q[action] += (reward - self.Q[action]) / self.N[action]
        
    def simulate(self, n_steps):
        rewards = np.zeros(n_steps)
        for i in range(n_steps):
            action = self.choose_action()
            reward = np.random.binomial(1, self.bandits[action])
            rewards[i] = reward
            self.update(action, reward)
        return rewards

In [2]:
import numpy as np
#今回はアームが4つの場合を想定します，
class TS_bandit:
    def __init__(self,alpha: float,beta: float):
        '''
        p -> それぞれ報酬が発生する確率
        alpha,beta -> ベータ分布の確率
        m -> それぞれの報酬が出た回数
        n -> ぞれぞれが試行された数
        '''
        self.p = [0.1, 0.3, 0.5, 0.8] #最適なアームのインデックスは3
        self.alpha = alpha
        self.beta = beta
        self.m = np.zeros(len(self.p))
        self.n = np.zeros(len(self.p))


    def choose_action(self,m: int,n: int):
        
        best_index = None
        prev_sample = 0
        for i in range(len(self.p)):
            sample = np.random.beta(self.alpha + m[i], self.beta + n[i] - m[i], 1)
            if best_index == None:
                best_index = i
                prev_sample = sample
                continue

            if sample > prev_sample:
                best_index = i
                prev_sample = sample

        return best_index

    def reward(self,index: int):
        return np.random.binomial(1,self.p[index],1)

    
    def update(self,index,reward):
        self.m[index] += reward
        self.n[index] += 1

    def simulate(self,N):
        for i in range(N):
            arm = self.choose_action(self.m,self.n)
            reward = self.reward(arm)
            self.update(arm,reward)


In [3]:
ban = TS_bandit(1,1) #今回は一様分布で実験
ban.simulate(1000)

print(f'0が選ばれた回数:{ban.n[0]}回, 1が選ばれた回数:{ban.n[1]}回, 2が選ばれた回数:{ban.n[2]}回, 3が選ばれた回数:{ban.n[3]}回')

0が選ばれた回数:6.0回, 1が選ばれた回数:4.0回, 2が選ばれた回数:7.0回, 3が選ばれた回数:983.0回


上の結果を見ればわかるように最適なアーム3が1番惹かれていることがわかりますね．